# Premiers tests dans l'implémentation de la méthode ROME.

### Résumé de ce que j'ai compris de l'article

La Méthode ROME fonctionne en gros sur un système key-value qu'on retrouve dans les poids du modèle. On assaie donc d'insérer un couple (k*,v*) à un endroit précis dans notre réseau de neurone.
En pratique:
1. On séléctionne la clé k en faisant une moyenne sur des textes tirés de Wikipédia, j'avoue que je saisis pas trop comment encore.
2. On résoud un problème d'optimisation afin de trouver v associé, voir l'équation sur l'article
3. On insère v* ainsi obtenu.
-> L'insertion se fait visiblement à la couche 18, puisque c'est là qu'on remarque un changement du comportement des couches de MLP qui switchent et commence à agir comme des valeurs (v) et non plus comme des clés (k)

Ils commencent par calculer les moments d'ordre 2 (voir les maths dans l'article) en utilisant 10000 échantillons de couches cachées (k), que l'on compute en utilisant des tokesnde toutes les pages Wikipedia (2020-05-01 snapshot)-> On ne se limite pas aux sujets mais tous les tokens sont aussi inclus visiblement.
-> Le text complet de chaque article est passé dans le transformer jusq'à ce qu'on atteigne la limite, puis on stocke les activations de la partie MLP (k) pour chaque token de l'article en question.
-> Et on repète le process jusqu'à obtenir 10 000 vecteur k. Ensuite on calcule les moments d'ordre 2 E[kk^T]

Bon c'est pas hyper clair mdr


## Etape 1: On essaie de compute les moments d'ordre 2 à partir de Wikipedia

In [27]:
from datasets import load_dataset

version = 'wikitext-103-raw-v1'

ds = load_dataset("wikitext", version)
ds

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [17]:
ds["validation"][:30]

{'text': ['',
  ' = Homarus gammarus = \n',
  '',
  ' Homarus gammarus , known as the European lobster or common lobster , is a species of clawed lobster from the eastern Atlantic Ocean , Mediterranean Sea and parts of the Black Sea . It is closely related to the American lobster , H. americanus . It may grow to a length of 60 cm ( 24 in ) and a mass of 6 kilograms ( 13 lb ) , and bears a conspicuous pair of claws . In life , the lobsters are blue , only becoming " lobster red " on cooking . Mating occurs in the summer , producing eggs which are carried by the females for up to a year before hatching into planktonic larvae . Homarus gammarus is a highly esteemed food , and is widely caught using lobster pots , mostly around the British Isles . \n',
  '',
  ' = = Description = = \n',
  '',
  ' Homarus gammarus is a large crustacean , with a body length up to 60 centimetres ( 24 in ) and weighing up to 5 – 6 kilograms ( 11 – 13 lb ) , although the lobsters caught in lobster pots are usua

Ensuite les auteurs virent les articles de moins de 50 caractères et ceux qui contiennent des caractères qui ne sont pas en ASCII
(j'ai volé leur fonction parce que flemme)

In [18]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def filter_text(iterator):
    valid = []
    for text in iterator:
        if len(text.split(" ")) < 50:
            continue
        if not is_ascii(text):
            continue
        valid.append(text)

    return valid

In [ ]:
clean_ds = filter_text(ds)
clean_ds

#Bon de fait juste copier leurs fonctions ça ne marche pas du tout, il va falloir faire ça nous même...

[]